In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from segment_mapping import *
import matplotlib.pyplot as plt
from apperception.utils import fetch_camera_config, fetch_camera_trajectory

In [2]:
test_config = fetch_camera_config('samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385110412404.jpg', database)

In [3]:
test_config

{'cameraId': 'scene-0655',
 'egoTranslation': [1921.691, 869.69446, 0.0],
 'egoRotation': [-0.9997083, 0.017253358, -0.006183327, -0.015729848],
 'cameraTranslation': [1.7220057, 0.004754533, 1.4949129],
 'cameraRotation': [0.50772417, -0.49733922, 0.49837166, -0.4964832],
 'cameraIntrinsic': [[1252.8131, 0.0, 826.58813],
  [0.0, 1252.8131, 469.98465],
  [0.0, 0.0, 1.0]],
 'frameNum': 2887,
 'fileName': 'samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385110412404.jpg',
 'timestamp': datetime.datetime(2018, 8, 27, 8, 51, 50, 412404, tzinfo=datetime.timezone.utc),
 'cameraHeading': -87.597466,
 'egoHeading': -88.18536}

In [4]:
mapping = map_imgsegment_roadsegment(test_config)


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:39: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary
/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:76: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for intersect in intersection:


total mapping time:  0.9228532314300537


In [5]:
test_file_path = '/home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385110412404.jpg'
# visualization(test_file_path, test_config, mapping)

In [6]:
%cd ../Yolov5_StrongSORT_OSNet
import sample_frame_tracker
full_img_detection = sample_frame_tracker.run(test_file_path, save_vid=True, detect_only=True)
full_img_detection

/home/yongming/workspace/research/apperception_new_local/apperception/Yolov5_StrongSORT_OSNet


strong_sort/deep/reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/yongming/.pyenv/versions/apperception_venv/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
YOLOv5 🚀 2022-8-20 Python-3.8.3 torch-1.12.1+cu102 CPU

Fusing layers... 


strong_sort/configs/strong_sort.yaml


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
[97.5, 514.0, 129.0, 54.0] 0.6214630603790283 2.0
[720.5, 481.5, 89.0, 71.0] 0.5629765391349792 2.0
[1032.0, 474.5, 98.0, 39.0] 0.25762975215911865 2.0


{1: single_detection_result(object_class='car', bbox=[97.5, 514.0, 129.0, 54.0]),
 2: single_detection_result(object_class='car', bbox=[720.5, 481.5, 89.0, 71.0]),
 3: single_detection_result(object_class='car', bbox=[1032.0, 474.5, 98.0, 39.0])}

In [7]:
import cv2
test_frame = cv2.imread(test_file_path)
for obj_idx, detection in full_img_detection.items():
    obj_cls, bbox = detection
    if obj_cls == 'car':
        x,y,w,h = list(map(int,bbox))
        cv2.rectangle(test_frame,(x-w//2,y-h//2),(x+w//2,y+h//2),(0,255,0),2)
        cv2.putText(test_frame, '_'.join([obj_cls, str(obj_idx)]), (x+w//2+5,y+h//2+5),0,0.3,(0,255,0))
cv2.imshow('detection', test_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
from detection_estimation.utils import *
import matplotlib.pyplot as plt

In [9]:
# ego car trajectory
test_video = 'CAM_FRONT/n008-2018-08-27'
ego_trajectory = get_ego_trajectory(test_video)

In [10]:
fetch_camera_trajectory(test_video, database)

[{'cameraId': 'scene-0655',
  'egoTranslation': [1772.1438, 866.30286, 0.0],
  'frameNum': 2,
  'timestamp': datetime.datetime(2018, 8, 27, 8, 51, 32, 112404, tzinfo=datetime.timezone.utc),
  'fileName': 'samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092112404.jpg',
  'cameraHeading': -88.09923,
  'egoHeading': -88.687126},
 {'cameraId': 'scene-0655',
  'egoTranslation': [1772.5153, 866.31036, 0.0],
  'frameNum': 13,
  'timestamp': datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc),
  'fileName': 'sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092162404.jpg',
  'cameraHeading': -88.078545,
  'egoHeading': -88.66644},
 {'cameraId': 'scene-0655',
  'egoTranslation': [1773.2598, 866.32367, 0.0],
  'frameNum': 27,
  'timestamp': datetime.datetime(2018, 8, 27, 8, 51, 32, 262404, tzinfo=datetime.timezone.utc),
  'fileName': 'sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092262404.jpg',
  'cameraHeading':

In [11]:
fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')
axs.plot([point.coordinates[0] for point in ego_trajectory], 
         [point.coordinates[1] for point in ego_trajectory], color='red', marker='o', markersize=2)


In [12]:
ego_speed = get_ego_speed(ego_trajectory)
ego_speed

[temporal_speed(speed=7.431513977648862, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 112404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=7.446189670563231, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=7.5621573833139175, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 262404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=7.52487880300925, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 362404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=7.608693005372625, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 412404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=7.9693059854657315, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 512404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=8.033426788117646, timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 612404, tzinfo=datetime.timezone.utc)),
 temporal_speed(speed=8.017304107990489, timestamp=datetime.datetime

In [13]:
[time_to_nearest_frame(test_video, point.timestamp) for point in ego_trajectory]

[('samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092112404.jpg',
  2,
  datetime.datetime(2018, 8, 27, 8, 51, 32, 112404, tzinfo=datetime.timezone.utc)),
 ('sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092162404.jpg',
  13,
  datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc)),
 ('sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092262404.jpg',
  27,
  datetime.datetime(2018, 8, 27, 8, 51, 32, 262404, tzinfo=datetime.timezone.utc)),
 ('sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092362404.jpg',
  41,
  datetime.datetime(2018, 8, 27, 8, 51, 32, 362404, tzinfo=datetime.timezone.utc)),
 ('sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092412404.jpg',
  51,
  datetime.datetime(2018, 8, 27, 8, 51, 32, 412404, tzinfo=datetime.timezone.utc)),
 ('sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092512404.jpg',
  66,
  datetime.datetime(2018, 8, 27, 8

In [14]:
import datetime
test_timestamp = datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc)
timestamp_to_nearest_trajectory(ego_trajectory, test_timestamp)

trajectory_3d(coordinates=[1772.5153, 866.31036, 0.0], timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc))

In [15]:
test_point = [1772, 865, 0.0]
point_to_nearest_trajectory(test_point, ego_trajectory)

trajectory_3d(coordinates=[1772.1438, 866.30286, 0.0], timestamp=datetime.datetime(2018, 8, 27, 8, 51, 32, 112404, tzinfo=datetime.timezone.utc))

In [16]:
from detection_estimation.sample_plan_algorithms import *

In [17]:
base_dir = '/home/yongming/workspace/research/apperception_new_local/boston-seaport'
test_img_base_dir = '/home/yongming/workspace/research/apperception/v1.0-mini/'

In [18]:
import pickle
with open(os.path.join(base_dir, f'frames.pickle'), "rb") as f:
    df_sample_data = pickle.loads(f.read())
# df_sample_data
# i = 0
# for frame in df_sample_data['scene-0655-CAM_FRONT']['frames']:
#     if frame[2] == 2887:
#         print(i)
#         break
#     i += 1

In [19]:
from detection_estimation.detection_estimation import *
# ### Integration
# target_config_idx = 218
# video = 'scene-0655-CAM_FRONT'
# configs = df_sample_data[video]
# sorted_ego_config = [dict(zip(configs['columns'], frame))
#                      for frame in configs['frames']]
# len(sorted_ego_config)
# # all_car_loc3d = 

In [20]:
# current_ego_config = sorted_ego_config[target_config_idx]

In [21]:
from shapely.geometry import Point, Polygon
current_ego_config = test_config
cam_segment_mapping = mapping
# visualization(test_file_path, current_ego_config, cam_segment_mapping)
# print(cam_segment_mapping)
current_frame = test_file_path
x,y,w,h = list(map(int, full_img_detection[2][1]))
car_loc2d = (x, y+h//2)
all_detections = []
for obj_idx, detection in full_img_detection.items():
    obj_cls, bbox = detection
    x,y,w,h = list(map(int,bbox))
    car_loc2d = (x,y+h//2)
    car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
    car_bbox3d = None
    estimate_3d = detection_to_img_segment(car_loc2d, cam_segment_mapping)
    if estimate_3d:
        car_loc3d = tuple(Polygon(estimate_3d.road_segment_info.segment_polygon).centroid.coords)
        print(tuple(car_loc3d))
        all_detections.append(obj_detection(obj_cls+str(obj_idx), car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
all_detection_info = construct_all_detection_info(
    current_frame, cam_segment_mapping, ego_trajectory, current_ego_config, all_detections)

((1972.356772971168, 874.6494952773176),)
91.14597135080668 -88.18536


In [22]:
next_sample_plan = generate_sample_plan(test_video, all_detection_info,  50)

ego_exit_segment_action action type: ego_exit_segment,
        start time: 2018-08-27 08:51:50.412404+00:00,
        finish time: 2018-08-27 08:51:50.512665+00:00,
        start loc: [1921.691, 869.69446, 0.0],
        end loc: [1922.5712, 869.7261, 0.0]
car_exit_segment_action action type: car_exit_segment,
        start time: 2018-08-27 08:51:50.412404+00:00,
        finish time: 2018-08-27 08:51:50.505342+00:00,
        start loc: ((1972.356772971168, 874.6494952773176),),
        end loc: (1972.319381286564, 876.5187392408118)
meet_ego_action action type: meet_up,
        start time: 2018-08-27 08:51:50.412404+00:00,
        finish time: 2018-08-27 08:51:51.677702+00:00,
        start loc: ((1972.356772971168, 874.6494952773176),),
        end loc: POINT (1947.0238864855842 872.1719776386587)


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:39: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


In [23]:
frame_name, frame_num, timestamp = next_sample_plan.get_next_frame_info()
print("next frame name", frame_name)
print("next frame num", frame_num)

next frame name sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385111662404.jpg
next frame num 3085


In [24]:
next_sample_plan.action

action type: meet_up,
        start time: 2018-08-27 08:51:50.412404+00:00,
        finish time: 2018-08-27 08:51:51.677702+00:00,
        start loc: ((1972.356772971168, 874.6494952773176),),
        end loc: POINT (1947.0238864855842 872.1719776386587)

In [25]:
next_frame = cv2.imread(test_img_base_dir+frame_name)
cv2.imshow("next_frame", next_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()